In [1]:
import pickle
import gc

with open('./merged_sorted.pkl', 'rb') as f:
    documents = pickle.load(f)


IOStream.flush timed out


In [7]:
%pip install ir_datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
import ir_datasets

dataset = ir_datasets.load("msmarco-passage/dev")

queries_count = sum(1 for _ in dataset.queries_iter())
print(f"Number of queries: {queries_count}")


Number of queries: 101093


In [9]:
print("=== Примеры релевантных пар query-passage ===")
for i, qrel in enumerate(dataset.qrels_iter()):
    if i >= 10:  # Покажем первые 10
        break
    print(f"Query {qrel.query_id} -> Passage {qrel.doc_id}, Relevance: {qrel.relevance}")

=== Примеры релевантных пар query-passage ===
Query 1102432 -> Passage 2026790, Relevance: 1
Query 1102431 -> Passage 7066866, Relevance: 1
Query 1102431 -> Passage 7066867, Relevance: 1
Query 1090282 -> Passage 7066900, Relevance: 1
Query 39449 -> Passage 7066905, Relevance: 1
Query 76162 -> Passage 7066915, Relevance: 1
Query 195512 -> Passage 7066971, Relevance: 1
Query 1090280 -> Passage 7067004, Relevance: 1
Query 331318 -> Passage 5309290, Relevance: 1
Query 300674 -> Passage 7067032, Relevance: 1


In [10]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.splade.models.transformer_rep import Splade

model_type_or_dir = "naver/splade-cocondenser-ensembledistil"

model = Splade(model_type_or_dir, agg="max")
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can 

In [13]:
device

device(type='cpu')

In [68]:
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

qrels_mapping = {}
for qrel in dataset.qrels_iter():
    qrels_mapping.setdefault(qrel.query_id, {})[qrel.doc_id] = qrel.relevance
    
import numpy as np
import heapq

def search_top10(query_bow_rep, optimized_posting_lists, top_k=10):
    all_doc_ids = []
    all_weights = []
    
    for term, weight in query_bow_rep:
        if term in optimized_posting_lists:
            doc_ids, scores = optimized_posting_lists[term]
            all_doc_ids.append(doc_ids)
            all_weights.append(scores * weight)
    
    if not all_doc_ids:
        return []
    
    all_doc_ids = np.concatenate(all_doc_ids)
    all_weights = np.concatenate(all_weights)
    
    unique_docs, inverse_indices = np.unique(all_doc_ids, return_inverse=True)
    doc_scores = np.zeros_like(unique_docs, dtype=np.float32)
    
    np.add.at(doc_scores, inverse_indices, all_weights)
    
    if len(doc_scores) <= top_k:
        top_indices = np.argsort(doc_scores)[::-1]
    else:
        top_indices = np.argpartition(doc_scores, -top_k)[-top_k:]
        top_indices = top_indices[np.argsort(doc_scores[top_indices])[::-1]]
    
    return [(doc_scores[i], unique_docs[i]) for i in top_indices]

mrr = 0
count = 0
responses = []
nullable = 0
for query in tqdm(dataset.queries_iter()):
    count += 1
    if count > 5000:
        break
    query_id = query.query_id
    relevant_passages = qrels_mapping.get(query_id, {})
    
    if len(relevant_passages) == 0:
        nullable += 1
    else:
        continue
    if len(relevant_passages) > 0:
        inputs = tokenizer(query.text, return_tensors="pt")
        inputs = {k: v for k, v in inputs.items()}

        query_rep = model(d_kwargs=inputs)["d_rep"].squeeze()

        nonzero_mask = query_rep != 0
        col = torch.nonzero(nonzero_mask).squeeze()
        weights = query_rep[col]

        sorted_weights, sorted_indices = torch.sort(weights, descending=True)
        sorted_col = col[sorted_indices]

        sorted_col_cpu = sorted_col.cpu().tolist()
        sorted_weights_cpu = sorted_weights.cpu().tolist()
            
        bow_rep = []
        for k, v in zip(sorted_col_cpu, sorted_weights_cpu):
            bow_rep.append((reverse_voc[k], v))
        
        # print('started')
        # Поиск топ-10 документов
        top10_docs = search_top10(bow_rep, optimized_posting_lists, top_k=10)
        # print(f"Query: {query.text}", )
        # print(f"Top-10 documents: {top10_docs}")
        found = False
        for index, doc in enumerate(top10_docs):
            doc_id = doc[1]
            if str(doc_id) in list(relevant_passages.keys()):
                responses.append(f"Query: {query.text} {index + 1}")
                mrr += 1 / (index + 1)
                found = True
                break
                
        if not found:
            responses.append(f"Query: {query.text} 0")
        
                
print(mrr / 5000)
        # print('finished')
        
print(mrr / 2781)

5000it [00:00, 375201.63it/s]

0.0
0.0


In [ ]:
with open('result.txt', 'w') as f:
    f.write(str({"a": mrr / 5000, "b": str(responses)}))
    f.close()


In [11]:
query

GenericQuery(query_id='1048578', text='cost of endless pools/swim spa')

In [2]:
import numpy as np
import cupy as cp
from tqdm import tqdm
import gc

def sort_dict_cuda_inplace(data_dict):
    keys_list = list(data_dict.keys())
    
    for key in tqdm(keys_list, desc="Sorting arrays"):
        arr = data_dict[key]
        
        if len(arr) == 0:
            continue
            
        values = np.array([x[1] for x in arr], dtype=np.float32)
        
        gpu_values = cp.asarray(values)
        gpu_indices = cp.argsort(-gpu_values)
        sorted_indices = cp.asnumpy(gpu_indices)
        
        sorted_arr = [arr[i] for i in sorted_indices]
        
        data_dict[key] = sorted_arr
        
        del gpu_values, gpu_indices, values, sorted_indices, arr
        cp.get_default_memory_pool().free_all_blocks()
    
    return data_dict


sorted_data = sort_dict_cuda_inplace(documents)


Sorting arrays: 100%|██████████| 27714/27714 [04:16<00:00, 108.25it/s] 


In [3]:
with open('./merged_sorted.pkl', 'wb') as f:
    pickle.dump(sorted_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
min(sorted_data['cat'], key=lambda x: x[1])

('136689', 0.010684899985790253)

In [ ]:
documents

27715

In [10]:
torch.tensor(doc_ids_list)

ValueError: too many dimensions 'str'

In [5]:
documents['cat'][50451]

('8549863', 1.310231328010559)

In [2]:
import pickle
import numpy as np
import sys
from tqdm import tqdm

original_size = sys.getsizeof(documents) / (1024**3)
print(f"Размер до: {original_size:.6f} ГБ")

optimized = {}
for term, posting_list in tqdm(documents.items()):
    optimized[term] = [(int(doc_id), np.float16(score)) for doc_id, score in posting_list]

optimized_size = sys.getsizeof(optimized) / (1024**3)
print(f"Размер после: {optimized_size:.6f} ГБ")
print(f"Экономия: {(1 - optimized_size/original_size)*100:.2f}%")

with open('optimized.pkl', 'wb') as f:
    pickle.dump(optimized, f)


Размер до: 0.001221 ГБ


 18%|█▊        | 5025/27714 [19:05<1:26:12,  4.39it/s]  


KeyboardInterrupt: 

In [5]:
import numpy as np
from tqdm import tqdm

def optimize_posting_lists(posting_lists):
    optimized = {}
    for term, docs_scores in tqdm(posting_lists.items()):
        doc_ids = np.array([x[0] for x in docs_scores], dtype=np.int32)
        scores = np.array([x[1] for x in docs_scores], dtype=np.float32)
        optimized[term] = (doc_ids, scores)
    return optimized

new_documents = optimize_posting_lists(documents)

100%|██████████| 27714/27714 [11:41<00:00, 39.52it/s]  


In [6]:
new_documents['cat']

(array([7733759, 4811806, 2758271, ..., 8480784, 8588158, 8822218],
       dtype=int32),
 array([3.1162891, 3.0853443, 3.0774562, ..., 0.0106849, 0.0106849,
        0.0106849], dtype=float32))

In [71]:
import ast
import json

with open("result.txt", "r") as file:
    file_content = file.read()

# 4. Parse using ast.literal_eval() (safer than eval)
parsed_data = ast.literal_eval(file_content)


import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# Extract numeric values from your objects
# Replace 'value' with the actual property name in your objects
old_values = list(map(lambda x: int(x.split(' ')[-1]), eval(parsed_data['b'])))

mrr = 0
average_mrr = 0
values = []
for i in range(len(old_values)):
    if old_values[i] != 0:
        mrr += 1 / old_values[i]
    average_mrr = mrr / (i + 1)
    values.append(average_mrr)

values[2780]

0.39208889117195533